In [1]:
# week5_6_task2_tabular.ipynb

In [2]:
!pip install -q google-generativeai pandas

In [4]:
import google.generativeai as genai
import pandas as pd
from google.colab import userdata # akses Kunci API dengan aman
import time

In [7]:
# Ambil Kunci API dari Colab Secrets
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        raise ValueError("Kunci API tidak ditemukan.")
    genai.configure(api_key=GEMINI_API_KEY)
    print("Kunci API Gemini berhasil dikonfigurasi")
except Exception as e:
    print(f"🚨 Waduh, ada error pas akses Kunci API: {e}")
    print("Pastikan kamu sudah mengatur GEMINI_API_KEY di Colab Secrets.")
    GEMINI_API_KEY = None # Jadi None kalau gagal diambil


Kunci API Gemini berhasil dikonfigurasi


In [9]:
# Inisialisasi Model LLM

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
]

if GEMINI_API_KEY:
    model = genai.GenerativeModel(
        model_name='gemini-1.5-flash',
        safety_settings=safety_settings
        # Bisa juga tambahkan generation_config di sini buat kontrol lebih
    )
    print("Model Gemini 1.5 Flash sudah siap dipakai")
else:
    print("Model nggak bisa disiapkan nih, Kunci API-nya nggak ada.")
    model = None

Model Gemini 1.5 Flash sudah siap dipakai


In [8]:
# Bikin dataset
data = {
    'id_produk': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'nama_produk': [
        'Lampu Surya EcoCahaya', 'Headset VR QuantumLompat', 'Drone SilumanX', 'Filter Air AquaMurni',
        'Pupuk Organik BioSubur', 'Speaker Bluetooth NovaSuara', 'Bantal Busa Memori AwanNyaman',
        'Blender Pro ChefHandal', 'Matras Yoga FlexiLentur', 'Casing HP TerraKuat',
        'Sepatu Lari AeroGesit', 'Teleskop BintangCemerlang', 'Keyboard Mekanik KodeKetik',
        'Botol Minum Anjing AquaGuk', 'Tablet E-Ink SunyiTulis'
    ],
    'deskripsi': [
        'Lampu luar ruangan hemat energi, terang benderang, pakai tenaga matahari. Tahan lama dan cuaca.',
        'Pengalaman realitas virtual yang bikin lupa dunia nyata, layar resolusi tinggi, kontrol gampang.',
        'Drone gesit dan ringkas dengan kamera 4K, cocok buat foto dan video dari atas.',
        'Sistem filter canggih multi-tahap yang buang kotoran, hasilin air minum bersih dan aman.',
        'Pupuk kaya nutrisi dari bahan alami buat tanaman tumbuh subur dan sehat.',
        'Speaker nirkabel portabel dengan bass mantap dan suara jernih, baterai tahan 12 jam.',
        'Bantal desain ergonomis yang ngikutin bentuk kepala dan leher, nyaman banget.',
        'Blender kecepatan tinggi dengan banyak setelan buat smoothie, sup, dan saus. Gampang dibersihin.',
        'Matras yoga anti-slip ramah lingkungan, empuk dan dukung semua pose yoga.',
        'Casing HP super kuat, tahan banting, dirancang buat lindungi HP dari jatuh dan benturan.',
        'Sepatu lari ringan dan responsif buat performa optimal dan nyaman di segala medan.',
        'Teleskop kuat buat pemula yang mau lihat bintang, planet, dan galaksi jauh dengan jelas.',
        'Keyboard taktil dan responsif buat programmer dan penulis, lampu RGB bisa diatur.',
        'Botol minum anti bocor buat anjing, pas buat jalan-jalan, hiking, dan travelling.',
        'Tablet e-ink bebas gangguan buat baca dan catat, layar kayak kertas.'
    ],
    'kategori': [
        'Rumah & Taman', 'Elektronik', 'Elektronik', 'Rumah & Taman',
        'Rumah & Taman', 'Elektronik', 'Rumah & Kasur',
        'Peralatan Dapur', 'Olahraga & Outdoor', 'Aksesoris',
        'Olahraga & Outdoor', 'Hobi', 'Aksesoris Komputer',
        'Perlengkapan Hewan', 'Elektronik'
    ]
}
df = pd.DataFrame(data)

# Tampilkan beberapa baris awal datasetnya
print("Dataset Asli:")
display(df.head())

Dataset Asli:


,id_produk,nama_produk,deskripsi,kategori
0,1,Lampu Surya EcoCahaya,"Lampu luar ruangan hemat energi, terang bender...",Rumah & Taman
1,2,Headset VR QuantumLompat,Pengalaman realitas virtual yang bikin lupa du...,Elektronik
2,3,Drone SilumanX,"Drone gesit dan ringkas dengan kamera 4K, coco...",Elektronik
3,4,Filter Air AquaMurni,Sistem filter canggih multi-tahap yang buang k...,Rumah & Taman
4,5,Pupuk Organik BioSubur,Pupuk kaya nutrisi dari bahan alami buat tanam...,Rumah & Taman


# Merumuskan Prompt LLM untuk Tiap Baris
#
# ** Tugas LLM ** Untuk setiap produk, buatkan tagline pemasaran yang singkat, menarik, dan gampang diingat (maksimal 10 kata) berdasarkan nama dan deskripsinya.

In [10]:
def buat_tagline(nama_produk, deskripsi_produk):
    """
    Fungsi ini minta LLM untuk membuat tagline pemasaran buat sebuah produk.
    """
    if not model:
        return "Error: Model LLM belum siap dipakai."

    prompt_llm = f"""
    Nama Produk: {nama_produk}
    Deskripsi Produk: {deskripsi_produk}

    Berdasarkan nama dan deskripsi produk di atas, tolong buatkan satu tagline pemasaran yang singkat dan menarik.
    Tagline harus maksimal 10 kata dan menonjolkan manfaat atau fitur utama produk.

    Tagline:
    """
    try:
        respons = model.generate_content(prompt_llm)
        # Bersihkan responsnya dikit kalau perlu
        tagline = respons.text.strip()
        # Pastikan nggak kepanjangan, walau prompt udah minta maks 10 kata
        if len(tagline.split()) > 15: # Kasih toleransi dikit dari 10 kata
            tagline = " ".join(tagline.split()[:12]) + "..."
        return tagline
    except Exception as e:
        return f"Error membuat tagline: {str(e)}"

# Iterasi per Baris dan Proses dengan LLM
#
# Ulang proses ini untuk sampel DataFrame (atau semua baris kalau datanya sedikit) dan terapkan fungsi `buat_tagline`. Hasilnya kita simpan di kolom baru.

In [11]:
# Pilih sampel dari dataframe (misalnya 10-20 baris pertama sesuai tugas)
# Untuk contoh ini, kita proses semua baris aja karena datanya kecil.
# Kalau datasetmu besar, pakai df_sample = df.head(20).copy()
df_hasil_proses = df.copy()
respons_llm_list = []

print("🚀 Mulai proses tiap baris dengan LLM...")

# Kalau kamu proses banyak baris dan khawatir kena limit API:
# for index, baris in df_hasil_proses.iterrows():
#     print(f"Memproses produk: {baris['nama_produk']}...")
#     tagline = buat_tagline(baris['nama_produk'], baris['deskripsi'])
#     respons_llm_list.append(tagline)
#     time.sleep(1) # Kasih jeda 1 detik biar aman dari limit API

# Buat dataset kecil, iterasi langsung nggak masalah:
for index, baris in df_hasil_proses.iterrows():
    print(f"Memproses produk: {baris['nama_produk']}...")
    tagline = buat_tagline(baris['nama_produk'], baris['deskripsi'])
    respons_llm_list.append(tagline)
    # Nggak perlu jeda buat dataset sekecil ini, tapi ingat ya buat data besar.

df_hasil_proses['tagline_buatan_llm'] = respons_llm_list
print("\n✅ Proses dengan LLM selesai!")

🚀 Mulai proses tiap baris dengan LLM...
Memproses produk: Lampu Surya EcoCahaya...
Memproses produk: Headset VR QuantumLompat...
Memproses produk: Drone SilumanX...
Memproses produk: Filter Air AquaMurni...
Memproses produk: Pupuk Organik BioSubur...
Memproses produk: Speaker Bluetooth NovaSuara...
Memproses produk: Bantal Busa Memori AwanNyaman...
Memproses produk: Blender Pro ChefHandal...
Memproses produk: Matras Yoga FlexiLentur...
Memproses produk: Casing HP TerraKuat...
Memproses produk: Sepatu Lari AeroGesit...
Memproses produk: Teleskop BintangCemerlang...
Memproses produk: Keyboard Mekanik KodeKetik...
Memproses produk: Botol Minum Anjing AquaGuk...
Memproses produk: Tablet E-Ink SunyiTulis...

✅ Proses dengan LLM selesai!


In [12]:
print("\nDataset Hasil Proses dengan Tagline Buatan LLM:")
display(df_hasil_proses)


Dataset Hasil Proses dengan Tagline Buatan LLM:


,id_produk,nama_produk,deskripsi,kategori,tagline_buatan_llm
0,1,Lampu Surya EcoCahaya,"Lampu luar ruangan hemat energi, terang bender...",Rumah & Taman,"EcoCahaya: Terang benderang, hemat energi, dan..."
1,2,Headset VR QuantumLompat,Pengalaman realitas virtual yang bikin lupa du...,Elektronik,"QuantumLompat: Rasakan realitas virtual, tanpa..."
2,3,Drone SilumanX,"Drone gesit dan ringkas dengan kamera 4K, coco...",Elektronik,"SilumanX: 4K udara, tangkapan spektakuler, ri..."
3,4,Filter Air AquaMurni,Sistem filter canggih multi-tahap yang buang k...,Rumah & Taman,"AquaMurni: Air bersih, sehat, dan aman, setiap..."
4,5,Pupuk Organik BioSubur,Pupuk kaya nutrisi dari bahan alami buat tanam...,Rumah & Taman,"BioSubur: Suburkan tanaman, panen melimpah!"
5,6,Speaker Bluetooth NovaSuara,Speaker nirkabel portabel dengan bass mantap d...,Elektronik,"NovaSuara: Bass mantap, suara jernih, 12 jam n..."
6,7,Bantal Busa Memori AwanNyaman,Bantal desain ergonomis yang ngikutin bentuk k...,Rumah & Kasur,"AwanNyaman: Tidur nyenyak, leher dan kepala ri..."
7,8,Blender Pro ChefHandal,Blender kecepatan tinggi dengan banyak setelan...,Peralatan Dapur,"Blender Pro ChefHandal: Cepat, mudah, hasil se..."
8,9,Matras Yoga FlexiLentur,"Matras yoga anti-slip ramah lingkungan, empuk ...",Olahraga & Outdoor,"FlexiLentur: Aliran yoga sempurna, nyaman, da..."
9,10,Casing HP TerraKuat,"Casing HP super kuat, tahan banting, dirancang...",Aksesoris,"TerraKuat: Lindungi HP-mu, tanpa kompromi keku..."
